In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Variables
nInst = 0
nt = 0
commRate = 0.0010
dirPosLimit = 10000

# Load prices function
def loadPrices(fn):
    global nt, nInst
    df = pd.read_csv(fn, sep='\s+', header=None, index_col=None)
    (nt, nInst) = df.shape
    return (df.values).T

# File path
pricesFile = "./prices.txt"
prcAll = loadPrices(pricesFile)

# My method below
def getMyPosition(prcSoFar):
    global currentPos
    #vars
    lookback = 20
    gamma = 0.8
    lower_threshold = 25
    upper_threshold = 75
    if prcSoFar.shape[1] < lookback:
        return np.zeros(prcSoFar.shape[0])
    
    gains = np.maximum(prcSoFar[:, 1:] - prcSoFar[:, :-1], 0)
    losses = np.maximum(prcSoFar[:, :-1] - prcSoFar[:, 1:], 0)
    avg_gain = np.mean(gains[:, -lookback:], axis=1)
    avg_loss = np.mean(losses[:, -lookback:], axis=1)
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    positions = np.zeros(prcSoFar.shape[0])
    
    for i in range(prcSoFar.shape[0]):
        if rsi[i] < lower_threshold:
            positions[i] = 100  # Buy signal
        elif rsi[i] > upper_threshold:
            positions[i] = -100  # Sell signal
    
    currentPos = gamma * positions
    return positions


# Don't touch this
def calcPL(prcHist):
    cash = 0
    curPos = np.zeros(nInst)
    totDVolume = 0
    value = 0
    #Var
    update_interval = 15
    todayPLL = []
    (_, nt) = prcHist.shape
    for t in range(50, 500, update_interval):
        prcHistSoFar = prcHist[:, :t]
        newPosOrig = getMyPosition(prcHistSoFar)
        curPrices = prcHistSoFar[:, -1]
        posLimits = np.array([int(x) for x in dirPosLimit / curPrices])
        newPos = np.clip(newPosOrig, -posLimits, posLimits)
        deltaPos = newPos - curPos
        dvolumes = curPrices * np.abs(deltaPos)
        dvolume = np.sum(dvolumes)
        totDVolume += dvolume
        comm = dvolume * commRate
        cash -= curPrices.dot(deltaPos) + comm
        curPos = np.array(newPos)
        posValue = curPos.dot(curPrices)
        todayPL = cash + posValue - value
        todayPLL.append(todayPL)
        value = cash + posValue
        ret = 0.0
        if totDVolume > 0:
            ret = value / totDVolume
        print("Day %d value: %.2lf todayPL: $%.2lf $-traded: %.0lf return: %.5lf" %
              (t, value, todayPL, totDVolume, ret))
    pll = np.array(todayPLL)
    (plmu, plstd) = (np.mean(pll), np.std(pll))
    annSharpe = 0.0
    if plstd > 0:
        annSharpe = np.sqrt(250) * plmu / plstd
    return (plmu, ret, plstd, annSharpe, totDVolume)

(meanpl, ret, plstd, sharpe, dvol) = calcPL(prcAll)
score = meanpl - 0.1*plstd
print("=====")
print("mean(PL): %.1lf" % meanpl)
print("return: %.5lf" % ret)
print("StdDev(PL): %.2lf" % plstd)
print("annSharpe(PL): %.2lf " % sharpe)
print("totDvolume: %.0lf " % dvol)
print("Score: %.2lf" % score)

Day 50 value: -11.68 todayPL: $-11.68 $-traded: 11683 return: -0.00100
Day 65 value: -44.26 todayPL: $-32.58 $-traded: 45259 return: -0.00098
Day 80 value: -166.99 todayPL: $-122.73 $-traded: 83993 return: -0.00199
Day 95 value: 8.23 todayPL: $175.23 $-traded: 100767 return: 0.00008
Day 110 value: 27.49 todayPL: $19.26 $-traded: 137505 return: 0.00020
Day 125 value: 177.73 todayPL: $150.24 $-traded: 183268 return: 0.00097
Day 140 value: 151.61 todayPL: $-26.12 $-traded: 197387 return: 0.00077
Day 155 value: 122.15 todayPL: $-29.46 $-traded: 226847 return: 0.00054
Day 170 value: 399.54 todayPL: $277.39 $-traded: 258455 return: 0.00155
Day 185 value: 376.91 todayPL: $-22.64 $-traded: 281091 return: 0.00134
Day 200 value: 288.85 todayPL: $-88.06 $-traded: 308152 return: 0.00094
Day 215 value: 407.87 todayPL: $119.03 $-traded: 330126 return: 0.00124
Day 230 value: 679.82 todayPL: $271.94 $-traded: 360181 return: 0.00189
Day 245 value: 634.37 todayPL: $-45.45 $-traded: 407629 return: 0.0015

<>:19: SyntaxWarning: invalid escape sequence '\s'
<>:19: SyntaxWarning: invalid escape sequence '\s'
C:\Users\jorda\AppData\Local\Temp\ipykernel_12372\4151768054.py:19: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv(fn, sep='\s+', header=None, index_col=None)


In [12]:
count = 0
for i in range(0,20,2):
    count += 1
    print(count)

1
2
3
4
5
6
7
8
9
10
